In [6]:
import os
import pickle
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

In [7]:
def ncVaraible_to_numpy(var):
    # type(var) == netCDF4._netCDF4.Variable
    value = var[:]
    mask = (value < -99999.98) & (value > -100000.00)
    value = np.ma.MaskedArray(value, mask=mask).data
    return value


In [8]:
def ipixload(cdf_path, pol='vv', adc_like_I=None, adc_like_Q=None, adc_cross_I=None, adc_cross_Q=None):
    """
    python script to parse IPIX radar dataset.
    reference: http://soma.ece.mcmaster.ca/ipix/dartmouth/cdfhowto.html

    """

    #read raw data
    rootgrp = nc.Dataset(cdf_path, "r")
    rng_bins = ncVaraible_to_numpy(rootgrp.variables['range'])
    adc_data = ncVaraible_to_numpy(rootgrp.variables['adc_data'])

    adc_like_I = ncVaraible_to_numpy(rootgrp.variables['adc_like_I']) if adc_like_I is None else adc_like_I
    adc_like_Q = ncVaraible_to_numpy(rootgrp.variables['adc_like_Q']) if adc_like_Q is None else adc_like_Q
    adc_cross_I = ncVaraible_to_numpy(rootgrp.variables['adc_cross_I']) if adc_cross_I is None else adc_cross_I
    adc_cross_Q = ncVaraible_to_numpy(rootgrp.variables['adc_cross_Q']) if adc_cross_Q is None else adc_cross_Q

    # extract desired polarization
    H_txpol = 0
    V_txpol = 1
    if pol == 'hh':
        xiq = adc_data[:,H_txpol,:,[adc_like_I, adc_like_Q]]
    elif pol == 'hv':
        xiq = adc_data[:, H_txpol, :, [adc_cross_I, adc_cross_Q]]
    elif pol == 'vv':
        xiq = adc_data[:, V_txpol, :, [adc_like_I, adc_like_Q]]
    elif pol == 'vh':
        xiq = adc_data[:, V_txpol, :, [adc_cross_I, adc_cross_Q]]
    else:
        raise Exception(' ')
    xiq = np.transpose(xiq, [2, 1, 0])

    # apply corrections to I and Q data, correction is applied per range bin
    iq_mean = np.expand_dims(np.mean(xiq, 1), 1)
    iq_std = np.expand_dims(np.std(xiq, 1), 1)
    xiq = (xiq - iq_mean) / iq_std
    I = xiq[:, :, 0]
    Q = xiq[:, :, 1]
    sin_beta = np.expand_dims(np.mean(I * Q, 1), 1)
    inbal = np.arcsin(sin_beta) * 180/np.pi
    I = (I - Q * sin_beta) / np.sqrt(1 - sin_beta ** 2)

    return I, Q, iq_mean, iq_std

In [9]:
def parse_ipix_data(cdf_path, pol, adc_like_I, adc_like_Q, adc_cross_I, adc_cross_Q):
    # assert config.f_c == np.expand_dims(ncVaraible_to_numpy(rootgrp.variables['RF_frequency']), 0)[0]*1e9
    rootgrp = nc.Dataset(cdf_path, "r")

    f_c = np.expand_dims(ncVaraible_to_numpy(rootgrp.variables['RF_frequency']), 0)[0]*1e9
    PRI = 1 / np.expand_dims(ncVaraible_to_numpy(rootgrp.variables['PRF']), 0)[0]
    rng_bins = ncVaraible_to_numpy(rootgrp.variables['range'])
    rng_bins = np.around(rng_bins, 1)
    B = (3e8 / (2 * np.diff(rng_bins)[0]))
    # load data
    I, Q, iq_mean, iq_std = ipixload(cdf_path, pol, adc_like_I, adc_like_Q, adc_cross_I, adc_cross_Q)
    iq_mean = np.squeeze(iq_mean, 1)
    iq_std = np.squeeze(iq_std, 1)
    X = np.expand_dims(iq_std[:,0], 1)*I + 1j*np.expand_dims(iq_std[:,1], 1)*Q

    return X, PRI, rng_bins, B


In [10]:

pol = 'hh'
# cdf_dir = '../datasets/IPIX/3m/cdf'
# pkl_dir = '../datasets/IPIX/3m/pkl/' + pol
cdf_dir = '/Users/arigra/Desktop/DL projects/Dafc/datasets/IPIX/15m/cdf'
pkl_dir = '/Users/arigra/Desktop/DL projects/Dafc/datasets/IPIX/15m/pkl1/' + pol
cdf_path_list = [f for f in os.listdir(cdf_dir) if not f.startswith('.')]

for cdf_path in cdf_path_list:
    print(cdf_path)

    cdf_path = os.path.join(cdf_dir, cdf_path)
    X, PRI, rng_bins, B = parse_ipix_data(cdf_path, pol=pol, adc_like_I=2, adc_like_Q=3, adc_cross_I=0, adc_cross_Q=1)
    # dump to pkl_file
    dir_name = pkl_dir
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    pkl_file_name = os.path.basename(cdf_path).replace(".CDF","") + "_pol_" + pol + ".pkl"

    pickle.dump({"PRI": PRI, "rng_bins": rng_bins, "B": B,"adc_data": X}, open(os.path.join(dir_name, pkl_file_name), 'wb'), protocol=3)

19980205_171203_ANTSTEP.CDF
19980205_175943_ANTSTEP.CDF
19980205_192053_ANTSTEP.CDF
19980205_180308_ANTSTEP.CDF
19980205_171654_ANTSTEP.CDF
19980205_175642_ANTSTEP.CDF
19980205_170935_ANTSTEP.CDF
19980205_180558_ANTSTEP.CDF
19980205_184733_ANTSTEP.CDF
19980205_190524_ANTSTEP.CDF
19980205_191043_ANTSTEP.CDF
19980205_171437_ANTSTEP.CDF
19980205_185111_ANTSTEP.CDF
19980205_184403_ANTSTEP.CDF
19980205_185421_ANTSTEP.CDF
19980205_184038_ANTSTEP.CDF
19980205_191536_ANTSTEP.CDF
19980205_183709_ANTSTEP.CDF
19980205_183327_ANTSTEP.CDF
